In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import plotly.express as px

In [31]:
#model = tf.keras.models.load_model('predict_model.keras')
model = pickle.load(open('linear_regressor.pkl', 'rb'))

In [32]:
scaler = pickle.load(open('scaler.pkl', 'rb'))

In [33]:
columns = ['CO(mg/m^3)_initial', 'Volume(m^3)', 'N_people_MA_prev_10min',	'Ambient-Air-Pump(L/min)', 'Ambient-Air-Pump_power(%)',	'Ambient-Air-Pump_number', 'CO(mg/m^3)_final']
#mae: 0.1278

In [34]:
output_column = 'CO(mg/m^3)_final'

In [35]:
input_columns = ['CO(mg/m^3)_initial', 'Volume(m^3)', 'N_people_MA_prev_10min',	'Ambient-Air-Pump(L/min)', 'Ambient-Air-Pump_power(%)',	'Ambient-Air-Pump_number']

In [36]:
CO = 0.018
volume = 30
people_now = 10
people_tn = [people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now]
pumps_l_min = 566
pump_power = 0.0
n_pumps = 6
index = 0

In [37]:
def create_df_input(CO, volume, people_tn, pumps_l_min, pump_power, n_pumps, index):
    df = pd.DataFrame({
        'CO(mg/m^3)_initial':CO,
        'Volume(m^3)':volume, 
        'N_people_MA_prev_10min': sum(people_tn) / len(people_tn),	
        'Ambient-Air-Pump(L/min)': pumps_l_min, 
        'Ambient-Air-Pump_power(%)': pump_power,
        'Ambient-Air-Pump_number': n_pumps
    }, index=[index])
    return df

In [38]:
df_sim = pd.DataFrame(columns = columns)
for i in range(10):
    index = i
    df_t = create_df_input(CO, volume, people_tn, pumps_l_min, pump_power, n_pumps, index)
    df_t_scaled = scaler.transform(df_t)
    df_t_scaled = pd.DataFrame(df_t_scaled, columns=input_columns)
    CO = model.predict(df_t_scaled)
    CO += (np.random.rand(1)[0]*2 -1)* 0.01 * CO
    df_t["CO(mg/m^3)_final"] = CO
    df_sim = pd.concat([df_sim,df_t])

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Ambient-Air-Pump(L/min)
- Ambient-Air-Pump_number
- Ambient-Air-Pump_power(%)
- CO(mg/m^3)_initial
- N_people_MA_prev_10min
- ...
Feature names seen at fit time, yet now missing:
- CO(mg/m^3)
- People


In [ ]:
fig = px.line(df_sim["CO(mg/m^3)_final"])
fig.show()

In [ ]:
df_sim

,CO(mg/m^3)_initial,Volume(m^3),N_people_MA_prev_10min,Ambient-Air-Pump(L/min),Ambient-Air-Pump_power(%),Ambient-Air-Pump_number,CO(mg/m^3)_final
0,0.018000,30,10.0,566,0.0,6,0.892840
1,0.892840,30,10.0,566,0.0,6,0.979588
2,0.979588,30,10.0,566,0.0,6,0.990196
3,0.990196,30,10.0,566,0.0,6,0.984265
4,0.984265,30,10.0,566,0.0,6,0.998204
5,0.998204,30,10.0,566,0.0,6,0.984346
6,0.984346,30,10.0,566,0.0,6,0.987731
7,0.987731,30,10.0,566,0.0,6,0.995314
8,0.995314,30,10.0,566,0.0,6,1.001593
9,1.001593,30,10.0,566,0.0,6,0.992096
